In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import re

from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model

from attention import Attention_layer

#import spacy
#from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.layers import Input,GRU, Dense, Dropout,Conv1D, Conv2D, LSTM,Reshape
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score


from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from wordcloud import WordCloud, ImageColorGenerator
from bs4 import BeautifulSoup

In [3]:
tweet=pd.read_csv('tweet_lemmatization.csv')
mbti=tweet['type']
tweet['E/I']=tweet['type'].apply(lambda s: s[0])
tweet['N/S']=tweet['type'].apply(lambda s: s[1])
tweet['F/T']=tweet['type'].apply(lambda s: s[2])
tweet['J/P']=tweet['type'].apply(lambda s: s[3])

In [4]:
X_train,X_test,y_train,y_test=train_test_split(tweet,mbti,test_size=0.1,random_state=42)

X_train=X_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

y_train_df=pd.DataFrame()
y_train_df['E/I']=y_train.apply(lambda x: int(x[0]=='E'))
y_train_df['S/N']=y_train.apply(lambda x: int(x[1]=='S'))
y_train_df['F/T']=y_train.apply(lambda x: int(x[2]=='F'))
y_train_df['P/J']=y_train.apply(lambda x: int(x[3]=='P'))
y_train_1=y_train_df['E/I'].values.reshape(-1,1)
y_train_2=y_train_df['S/N'].values.reshape(-1,1)
y_train_3=y_train_df['F/T'].values.reshape(-1,1)
y_train_4=y_train_df['P/J'].values.reshape(-1,1)


y_test_df=pd.DataFrame()
y_test_df['E/I']=y_test.apply(lambda x: int(x[0]=='E'))
y_test_df['S/N']=y_test.apply(lambda x: int(x[1]=='S'))
y_test_df['F/T']=y_test.apply(lambda x: int(x[2]=='F'))
y_test_df['P/J']=y_test.apply(lambda x: int(x[3]=='P'))
y_test_1=y_test_df['E/I'].values.reshape(-1,1)
y_test_2=y_test_df['S/N'].values.reshape(-1,1)
y_test_3=y_test_df['F/T'].values.reshape(-1,1)
y_test_4=y_test_df['P/J'].values.reshape(-1,1)

In [5]:
w2v = Word2Vec.load('tweet_w2v.model')

In [15]:
def vectorize_reviews(data, maxlen=1000, embedding_dim=100):
    """
    Tokenizes reviews, truncates the number of tokens if more than `maxlen`,
    and vectorizes each token. Returns a three-dimensional array of shape
    n reviews x `maxlen` x `embedding_dim`.
    """
    # Create empty array
    vectorized_data = np.zeros(shape=(len(data), maxlen, embedding_dim))

    for row, review in enumerate(data):
        # Preprocess each review
        tokens = simple_preprocess(review)

        # Truncate long reviews
        if len(tokens) > maxlen:
            tokens = tokens[:maxlen]

        # Get vector for each token in review
        for col, token in enumerate(tokens):
            try:
                word_vector = w2v.wv[token]
                # Add vector to array
                vectorized_data[row, col] = word_vector[:embedding_dim]
            except KeyError:
                pass

    return vectorized_data

maxlen = 1000       # Our predetermined limit
embedding_dim = 300 # The first 100 values in our w2v vectors

X_train = vectorize_reviews(X_train.posts, maxlen, embedding_dim)
X_test= vectorize_reviews(X_test.posts, maxlen, embedding_dim)


print('Shape of feature matrix:', X_train.shape)
print('Shape of feature matrix:', X_test.shape)

Shape of feature matrix: (7807, 1000, 300)
Shape of feature matrix: (868, 1000, 300)


In [1]:
attention_model3= Sequential([
    Input(shape=(maxlen, embedding_dim),name='input'),
    LSTM(64,return_sequences=True,name='lstm'),
    Attention_layer(units=128,name='attention'),
    Dense(16, activation='sigmoid',name='dense'),
    Dense(1, activation='sigmoid',name='output')
])

# print(attention_model.summary())

attention_model3.compile(loss='binary_crossentropy',
                  metrics=['accuracy'],
                  optimizer='adam')

history = attention_model3.fit(X_train, y_train_3, epochs=8,
                              validation_split=0.1)

NameError: name 'Sequential' is not defined

In [ ]:
import pickle

filename = 'model/model3.pkl'
pickle.dump(attention_model3, open(filename, 'wb'))

In [ ]:
model = pickle.load(open('model.pkl','rb'))